<h1> Text Classification using TensorFlow/Keras on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Using TF Hub for transfer learning
<li> Creating a sentence level text classification model using Keras
<li> Creating a word lelvel text classification model using Keras
</ol>

In [5]:
# change these to try this notebook out
BUCKET = 'vijays-sandbox-ml'
PROJECT = 'vijays-sandbox'
REGION = 'us-central1'
SEED = 0

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

In [ ]:
#!pip3 install tf-nightly-2.0-preview

In [3]:
import tensorflow as tf
print(tf.__version__) # tf 2.0 nightly

2.0.0-beta1


We will look at the titles of articles and figure out whether the article came from the New York Times, TechCrunch or GitHub. 

We will use [hacker news](https://news.ycombinator.com/) as our data source. It is an aggregator that displays tech related headlines from various  sources.

### Creating Dataset from BigQuery 

Hacker news headlines are available as a BigQuery public dataset. The [dataset](https://bigquery.cloud.google.com/table/bigquery-public-data:hacker_news.stories?tab=details) contains all headlines from the sites inception in October 2006 until October 2015. 

Here is a sample of the dataset:

In [ ]:
#%load_ext google.cloud.bigquery #appears to be pre loaded in ai platform notebook, remove once confirmed

In [ ]:
%%bigquery --project $PROJECT
SELECT
  url, title, score
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  LENGTH(title) > 10
  AND score > 10
  AND LENGTH(url) > 0
LIMIT 10

Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL. For example, if the url is http://mobile.nytimes.com/...., I want to be left with <i>nytimes</i>

In [ ]:
%%bigquery --project $PROJECT
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  COUNT(title) AS num_articles
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
GROUP BY
  source
ORDER BY num_articles DESC
LIMIT 10

Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for machine learning.

In [ ]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

query="""
SELECT source, LOWER(REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ')) AS title FROM
  (SELECT
    ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
    title
  FROM
    `bigquery-public-data.hacker_news.stories`
  WHERE
    REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
    AND LENGTH(title) > 10
  )
WHERE (source = 'github' OR source = 'nytimes' OR source = 'techcrunch')
"""

df = bq.query(query + " LIMIT 5").to_dataframe()
df.head()

For ML training, we will need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset).  

A simple, repeatable way to do this is to use the hash of a well-distributed column in our data (See https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning).

In [ ]:
traindf = bq.query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) > 0").to_dataframe()
evaldf  = bq.query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) = 0").to_dataframe()

Below we can see that roughly 75% of the data is used for training, and 25% for evaluation. 

We can also see that within each dataset, the classes are roughly balanced.

In [ ]:
traindf['source'].value_counts()

In [ ]:
evaldf['source'].value_counts()

Finally we will save our data, which is currently in-memory, to disk.

In [ ]:
import os, shutil
DATADIR='data/txtcls'
shutil.rmtree(DATADIR, ignore_errors=True)
os.makedirs(DATADIR)
traindf.to_csv( os.path.join(DATADIR,'train.tsv'), header=False, index=False, encoding='utf-8', sep='\t')
evaldf.to_csv( os.path.join(DATADIR,'eval.tsv'), header=False, index=False, encoding='utf-8', sep='\t')

In [ ]:
!head -3 data/txtcls/train.tsv

In [ ]:
!wc -l data/txtcls/*.tsv

# Sentence Level Model with DNN

In [7]:
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense

import pandas as pd
import numpy as np
from google.cloud import storage

CLASSES = {'github': 0, 'nytimes': 1, 'techcrunch': 2}  # label-to-int mapping
TOP_K = 20000  # Limit on the number vocabulary size used for tokenization
MAX_SEQUENCE_LENGTH = 50  # Sentences will be truncated/padded to this length

"""
Helper function to download data from Google Cloud Storage
  # Arguments:
      source: string, the GCS URL to download from (e.g. 'gs://bucket/file.csv')
      destination: string, the filename to save as on local disk. MUST be filename
      ONLY, doesn't support folders. (e.g. 'file.csv', NOT 'folder/file.csv')
  # Returns: nothing, downloads file to local disk
"""
def download_from_gcs(source, destination):
    search = re.search('gs://(.*?)/(.*)', source)
    bucket_name = search.group(1)
    blob_name = search.group(2)
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    bucket.blob(blob_name).download_to_filename(destination)


"""
Parses raw tsv containing hacker news headlines and returns (sentence, integer label) pairs
  # Arguments:
      train_data_path: string, path to tsv containing training data.
        can be a local path or a GCS url (gs://...)
      eval_data_path: string, path to tsv containing eval data.
        can be a local path or a GCS url (gs://...)
  # Returns:
      ((train_sentences, train_labels), (test_sentences, test_labels)):  sentences
        are lists of strings, labels are numpy integer arrays
"""
def load_hacker_news_data(train_data_path, eval_data_path):
    if train_data_path.startswith('gs://'):
        download_from_gcs(train_data_path, destination='train.csv')
        train_data_path = 'train.csv'
    if eval_data_path.startswith('gs://'):
        download_from_gcs(eval_data_path, destination='eval.csv')
        eval_data_path = 'eval.csv'

    # Parse CSV using pandas
    column_names = ('label', 'text')
    df_train = pd.read_csv(train_data_path, names=column_names, sep='\t')
    df_eval = pd.read_csv(eval_data_path, names=column_names, sep='\t')

    return ((list(df_train['text']), np.array(df_train['label'].map(CLASSES))),
            (list(df_eval['text']), np.array(df_eval['label'].map(CLASSES))))


"""
Create tf.estimator compatible input function
  # Arguments:
      texts: [strings], list of sentences
      labels: numpy int vector, integer labels for sentences
      batch_size: int, number of records to use for each train batch
      mode: tf.estimator.ModeKeys.TRAIN or tf.estimator.ModeKeys.EVAL 
  # Returns:
      tf.data.dataset, produces feature and label
        tensors one batch at a time
"""
def input_fn(texts, labels, batch_size, mode):
    # Transform text to sequence of integers
    labels = tf.one_hot(labels,len(CLASSES)) #precision and recall metrics require one hot labels
    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return dataset.batch(batch_size)
    else: 
        return dataset.shuffle(50000).batch(batch_size)

"""
Builds a CNN model using keras and converts to tf.estimator.Estimator
  # Arguments
      model_dir: string, file path where training files will be written
      config: tf.estimator.RunConfig, specifies properties of tf Estimator
      filters: int, output dimension of the layers.
      kernel_size: int, length of the convolution window.
      embedding_dim: int, dimension of the embedding vectors.
      dropout_rate: float, percentage of input to drop at Dropout layers.
      pool_size: int, factor by which to downscale input at MaxPooling layer.
      embedding_path: string , file location of pre-trained embedding (if used)
        defaults to None which will cause the model to train embedding from scratch
      word_index: dictionary, mapping of vocabulary to integers. used only if
        pre-trained embedding is provided

    # Returns
        A keras model
"""
def keras_model(learning_rate):
    # Create model instance.
    model = models.Sequential()

    # Add embedding layer
    hub_layer = hub.KerasLayer(
        "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1", 
        output_shape=[128], 
        input_shape=[], 
        dtype=tf.string
    )
    model.add(hub_layer)
    model.add(Dense(500,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(len(CLASSES), activation='softmax'))

    # Compile model with learning parameters.
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )

    return model

In [8]:
hparams = {
    'train_data_path':'./data/txtcls/train.tsv',
    'eval_data_path':'./data/txtcls/eval.tsv',
    'batch_size':128,
    'learning_rate':.001
}

# Load Data
((train_texts, train_labels), (test_texts, test_labels)) = load_hacker_news_data(
    hparams['train_data_path'], hparams['eval_data_path'])

model = keras_model(learning_rate=hparams['learning_rate'])

train_dataset = input_fn(
    train_texts,
    train_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.TRAIN
)
eval_dataset = input_fn(
    test_texts,
    test_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.EVAL
)

In [9]:
%%time
tf.random.set_seed(SEED)
model.fit(
    train_dataset,
    epochs=5,
    validation_data=eval_dataset,
    validation_steps=None
)

Epoch 1/5
564/564 [==============================] - 7s 12ms/step - loss: 0.5376 - accuracy: 0.7716 - precision_2: 0.8022 - recall_2: 0.7341 - val_loss: 0.5059 - val_accuracy: 0.7823 - val_precision_2: 0.8018 - val_recall_2: 0.7591
Epoch 2/5
564/564 [==============================] - 5s 9ms/step - loss: 0.4562 - accuracy: 0.8098 - precision_2: 0.8293 - recall_2: 0.7867 - val_loss: 0.4819 - val_accuracy: 0.7957 - val_precision_2: 0.8139 - val_recall_2: 0.7752
Epoch 3/5
564/564 [==============================] - 5s 9ms/step - loss: 0.4166 - accuracy: 0.8292 - precision_2: 0.8458 - recall_2: 0.8096 - val_loss: 0.4781 - val_accuracy: 0.8003 - val_precision_2: 0.8162 - val_recall_2: 0.7830
Epoch 4/5
564/564 [==============================] - 5s 9ms/step - loss: 0.3841 - accuracy: 0.8447 - precision_2: 0.8586 - recall_2: 0.8281 - val_loss: 0.4847 - val_accuracy: 0.7988 - val_precision_2: 0.8135 - val_recall_2: 0.7833
Epoch 5/5
564/564 [==============================] - 6s 11ms/step - loss: 0

We get 80% accuracy. Not bad, however this method was using sentence level embeddigns and ignoring the ordering of words. Would we get better performance if we embedded each word individually then fed them into a sequential model?

# Word Level Model with CNN

This model takes into account the order of words

In [12]:
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Conv1D
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D

"""
Create tf.estimator compatible input function
  # Arguments:
      texts: [strings], list of sentences
      labels: numpy int vector, integer labels for sentences
      tokenizer: tf.python.keras.preprocessing.text.Tokenizer
        used to convert sentences to integers
      batch_size: int, number of records to use for each train batch
      mode: tf.estimator.ModeKeys.TRAIN or tf.estimator.ModeKeys.EVAL 
  # Returns:
      tf.data.dataset, produces feature and label
        tensors one batch at a time
"""
def input_fn(texts, labels, batch_size, mode):
    #precision and recall metrics require one hot labels
    labels = tf.one_hot(labels,len(CLASSES)) 
    #split sentences into lists of words
    texts = [sentence.split() for sentence in texts] 
    # pad to constant length
    texts = [(sentence + MAX_SEQUENCE_LENGTH * ['<PAD>'])[:MAX_SEQUENCE_LENGTH] for sentence in texts] 
    #embed
    embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1")
    texts = [embed(sentence) for sentence in texts]

    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return dataset.batch(batch_size)
    else: 
        return dataset.shuffle(50000).batch(batch_size)

"""
Builds a CNN model using keras 
  # Arguments
      model_dir: string, file path where training files will be written
      config: tf.estimator.RunConfig, specifies properties of tf Estimator
      filters: int, output dimension of the layers.
      kernel_size: int, length of the convolution window.
      embedding_dim: int, dimension of the embedding vectors.
      dropout_rate: float, percentage of input to drop at Dropout layers.
      pool_size: int, factor by which to downscale input at MaxPooling layer.


    # Returns
        A tf.estimator.Estimator 
"""
def keras_model(learning_rate, filters=64, dropout_rate=0.2, kernel_size=3, pool_size=3):
    # Create model instance.
    model = models.Sequential()

    model.add(Dropout(input_shape=(MAX_SEQUENCE_LENGTH,128),rate=dropout_rate))
    model.add(Conv1D(
        filters=filters,
        kernel_size=kernel_size,
        activation='relu',
        bias_initializer='random_uniform',
        padding='same'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(
        filters=filters * 2,
        kernel_size=kernel_size,
        activation='relu',
        bias_initializer='random_uniform',
        padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(len(CLASSES), activation='softmax'))

    # Compile model with learning parameters.
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )
    return model

Note this will take several minutes because now we are doing a lot of pre-processing in the input function. In particular we are embedding each individual word and storing the vector representations.

In [13]:
%%time
train_dataset = input_fn(
    train_texts,
    train_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.TRAIN
)
eval_dataset = input_fn(
    test_texts,
    test_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.EVAL
)

CPU times: user 5min 4s, sys: 45.7 s, total: 5min 50s
Wall time: 4min 10s


In [15]:
%%time
model = keras_model(learning_rate=hparams['learning_rate'])

tf.random.set_seed(SEED)
model.fit(
    train_dataset,
    epochs=5,
    validation_data=eval_dataset,
    validation_steps=None
)

Epoch 1/5
564/564 [==============================] - 9s 17ms/step - loss: 0.6690 - accuracy: 0.7032 - precision_4: 0.7619 - recall_4: 0.6068 - val_loss: 0.5172 - val_accuracy: 0.7905 - val_precision_4: 0.8194 - val_recall_4: 0.7484
Epoch 2/5
564/564 [==============================] - 7s 13ms/step - loss: 0.4999 - accuracy: 0.7949 - precision_4: 0.8197 - recall_4: 0.7635 - val_loss: 0.4679 - val_accuracy: 0.8077 - val_precision_4: 0.8328 - val_recall_4: 0.7792
Epoch 3/5
564/564 [==============================] - 7s 13ms/step - loss: 0.4642 - accuracy: 0.8088 - precision_4: 0.8305 - recall_4: 0.7829 - val_loss: 0.4449 - val_accuracy: 0.8157 - val_precision_4: 0.8392 - val_recall_4: 0.7907
Epoch 4/5
564/564 [==============================] - 8s 13ms/step - loss: 0.4392 - accuracy: 0.8207 - precision_4: 0.8415 - recall_4: 0.7960 - val_loss: 0.4285 - val_accuracy: 0.8236 - val_precision_4: 0.8458 - val_recall_4: 0.8000
Epoch 5/5
564/564 [==============================] - 8s 14ms/step - loss

Our accuracy improved to 83%! Looks like paying attention to word order does help.

# Save Trained Model

In [16]:
model.save('headline_classification_model.h5')

# Deploy trained model 

See [deploy.ipynb](deploy.ipynb)

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License